In [15]:
import cv2
import torch
import numpy as np
import argparse



In [14]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'

In [22]:
from backbone import build_backbone, Backbone, Joiner
from position_encoding import build_position_encoding

In [17]:
train_backbone = False
return_interm_layers = False
dilation = False
backbone = Backbone('resnet18', train_backbone, return_interm_layers, dilation).to(DEVICE) 
backbone

Backbone(
  (body): IntermediateLayerGetter(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): FrozenBatchNorm2d()
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): FrozenBatchNorm2d()
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): FrozenBatchNorm2d()
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): FrozenBatchNorm2d()
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): FrozenBatchNorm2d()
      )
    )
    (layer2): Sequential(
   

In [18]:
# image = np.random.rand(518, 518, 3)
image = np.random.rand(480, 640, 3)

image = image.astype('float32')/255
images_arr = np.expand_dims(image, axis=0)
print(images_arr.shape)
input_tensor = torch.Tensor(np.transpose(images_arr, [0, 3, 2, 1])).to(DEVICE)



(1, 480, 640, 3)


In [20]:
result = backbone(input_tensor)


In [21]:
result['0'].shape

torch.Size([1, 512, 20, 15])

In [13]:

args = argparse.Namespace(hidden_dim=512, position_embedding='sine')

print(args.hidden_dim)  # Output: 512

512


In [16]:
position_embedding = build_position_encoding(args)
position_embedding

PositionEmbeddingSine()

In [23]:
net = Joiner(backbone, position_embedding).to(DEVICE)

In [24]:
result = net(input_tensor)


In [33]:
result[0][0].shape

torch.Size([1, 512, 20, 15])

In [35]:
result[1][0].shape

torch.Size([1, 512, 20, 15])